# Compare truck arrival distribution with results

In [ ]:
import os
import sys

import datetime

import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import pandas as pd
from scipy.stats import gmean

In [ ]:
path_to_conflowgen = os.path.abspath(
    os.path.join(
        os.pardir,  # notebooks
        os.pardir,  # tests
        os.pardir  # conflowgen
    )
)

path_to_conflowgen

In [ ]:
sys.path.insert(
    0,
    path_to_conflowgen
)

In [ ]:
import conflowgen

conflowgen.__file__

Load previously defined data.

In [ ]:
conflowgen.setup_logger(
    logging_directory="./data/logger",  # use subdirectory relative to Jupyter Notebook
)

database_chooser = conflowgen.DatabaseChooser(
    sqlite_databases_directory="../../data/databases"  # use subdirectory relative to Jupyter Notebook
)

In [ ]:
file_name = "demo_continental_gateway.sqlite"

database_chooser.load_existing_sqlite_database(file_name)

Visualize weekly curve.

In [ ]:
truck_arrival_distribution_manager = conflowgen.TruckArrivalDistributionManager()

truck_arrival_distribution = truck_arrival_distribution_manager.get_truck_arrival_distribution()

hour_in_week, fraction = zip(*list(sorted(truck_arrival_distribution.items())))
weekday_in_week = [x / 24 + 1 for x in hour_in_week]
percentage = [x * 100 for x in fraction]

fig, ax = plt.subplots(figsize=(15, 3))
plt.plot(weekday_in_week, percentage)
plt.xlim([1, 7])  # plot from Monday to Sunday
ax.xaxis.grid(True, which='minor', color='lightgray')  # every hour
ax.xaxis.grid(True, which='major', color='k')  # every day
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1 / 24))  # every hour

plt.title("Truck arrival distribution")
ax.set_xticks([i for i in range(1, 8)])  # every day
ax.set_xticklabels(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plt.xlabel("Week day")
plt.ylabel("Share (as percentage overall)")

plt.show()

In [ ]:
truck_gate_throughput_analysis_report = conflowgen.TruckGateThroughputAnalysisReport()
truck_gate_throughput_analysis_report

Check the reported data.

In [ ]:
truck_gate_throughput_analysis_report.show_report_as_graph()

Take a smaller slice.

In [ ]:
truck_gate_throughput_analysis_report.show_report_as_graph(
    start_date=datetime.datetime(2021, 6, 28),
    end_date=datetime.datetime(2021, 7, 5)
)

Also check the reported numbers.

In [ ]:
text = truck_gate_throughput_analysis_report.get_report_as_text(
    start_date=datetime.datetime(2021, 6, 28),
    end_date=datetime.datetime(2021, 7, 5)
)
print(text)

Project weekly curve on same time range.

In [ ]:
hour_in_week, fraction = zip(*list(sorted(truck_arrival_distribution.items())))
dates = [datetime.datetime(2021, 6, 28) + datetime.timedelta(hours=x) for x in hour_in_week]
percentage = [x * 100 for x in fraction]

fig, ax = plt.subplots(figsize=(15, 3))
plt.plot(dates, percentage)
ax.xaxis.grid(True, which='minor', color='lightgray')

plt.title("Truck arrival distribution")

plt.show()

Combine both visuals.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))

plt.plot(dates, np.array(percentage) * 50)  # use factor to scale this approximately to the same scale

truck_gate_throughput_analysis_report.get_report_as_graph(
    start_date=datetime.datetime(2021, 6, 28),
    end_date=datetime.datetime(2021, 7, 5)
)

plt.show()

In [ ]:
truck_gate_throughput_analysis_report.get_report_as_graph(
    start_date=datetime.datetime(2021, 6, 28),
    end_date=datetime.datetime(2021, 7, 31)
)

plt.show()

In [ ]:
conflowgen.run_all_previews(as_graph=True)

In [ ]:
conflowgen.run_all_analyses(as_graph=True)

In [ ]:
truck_gate_throughput_preview_report = conflowgen.TruckGateThroughputPreviewReport()
truck_gate_throughput_preview_report.show_report_as_graph()

truck_gate_throughput_analysis_report = conflowgen.TruckGateThroughputAnalysisReport()
truck_gate_throughput_analysis_report.show_report_as_graph()

In [ ]:
# Get analysis data
truck_gate_throughput_analysis = conflowgen.TruckGateThroughputAnalysis()
truck_gate_throughput_analysis_data = truck_gate_throughput_analysis.get_throughput_over_time()

# Convert the dictionary to a pandas Series
series_data = pd.Series(truck_gate_throughput_analysis_data)

# Set the datetime values as the index
series_data.index = pd.to_datetime(series_data.index)

# Filter the data for the period in July 2021
filtered_data = series_data.loc['2021-07-5':'2021-07-31']

# Group the data by week starting on Monday
weekly_grouped_data = filtered_data.groupby(pd.Grouper(freq="W-SUN"))

In [ ]:
# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

# Iterate over each group and plot the data
for i, (group_name, group_data) in enumerate(weekly_grouped_data):
    # Extract the start date of the week for labeling
    week_start = group_data.index[0].strftime('%Y-%m-%d')

    if i == 0:
        offset = 168 - len(group_data)
    else:
        offset = 0

    # Plot the data for the week
    ax.plot(range(offset, offset + len(group_data)), group_data.values, label=week_start, linestyle='--')


# Create a custom key function to combine the day of the week and hour of the day
def key_func(index):
    return (index.dayofweek * 24) + index.hour


# Group the data using the custom key function
hourly_grouped_data = filtered_data.groupby(key_func)

# Take average of grouped_data
average_grouped_data = hourly_grouped_data.apply(gmean)
average_week_range = range(0, len(average_grouped_data))  # Adjust the range to start from 0
ax.plot(average_week_range, average_grouped_data.values, label='Average', color='black', linestyle='solid')

# Plot preview
start_date = datetime.date(year=2021, month=7, day=5)
end_date = datetime.date(year=2021, month=7, day=31)
transportation_buffer = 0.2
truck_gate_throughput_preview = conflowgen.TruckGateThroughputPreview(start_date, end_date, transportation_buffer)
preview_data = truck_gate_throughput_preview.get_weekly_truck_arrivals(True, True)
preview_series = pd.Series(preview_data)
preview_series.index = pd.to_datetime(preview_series.index)
average_week_range = range(0, len(preview_series))
ax.plot(average_week_range, preview_series.values, label='Preview', color='yellow', linestyle='solid')

# Set x-axis tick labels
ax.set_xticks([12, 36, 60, 84, 108, 132, 156])  # Set the ticks at the middle of each day
ax.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# Add legend and labels
ax.legend(title='Week Start')
ax.set_xlabel('Day of the Week')
ax.set_ylabel('Truck Arrivals')

# Show the plot
plt.show(block=True)

print("Total weekly trucks according to preview: ", preview_series.sum())
print("Total weekly trucks according to average: ", average_grouped_data.sum())

In [ ]:
database_chooser.close_current_connection()